In [2]:
import keras
import csv
import os
os.chdir("C:/New/spam")
with open('emaildata1.csv' , 'r' ) as emaildata:
    email_reader = csv.reader(emaildata)
print("done")

done


In [3]:
import pandas as pd
train_data = pd.read_csv("C:/New/spam/train.csv")
#print (train_data)


got all labels(0,1)
matched the labels with the email text sr.no
total email - 2181

In [4]:
label = train_data.label.unique()
print(label)
#.unique gets all the unique values from the list
dic = {}
for i,label in enumerate(label):
    dic[label] = i
labels = train_data.label.apply(lambda x:dic[x]) #gave a numeric label to each email
y_train ={}
   
print(dic)
#type(y_train)
print(labels)


[0 1]
{0: 0, 1: 1}
0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         1
23         0
24         0
25         0
26         0
27         0
28         0
29         0
          ..
1048545    0
1048546    0
1048547    0
1048548    0
1048549    0
1048550    0
1048551    0
1048552    0
1048553    0
1048554    0
1048555    0
1048556    0
1048557    1
1048558    0
1048559    0
1048560    0
1048561    0
1048562    0
1048563    0
1048564    0
1048565    0
1048566    0
1048567    0
1048568    0
1048569    0
1048570    0
1048571    0
1048572    0
1048573    0
1048574    0
Name: label, Length: 1048575, dtype: int64


tokenized the emails (texts) using keras tokenizer
word index - a unqiue integer is given to every word
sequence_train is all the email data represented in integers 
total words in the texts is 40162

In [5]:
from keras.preprocessing.text import Tokenizer
texts = train_data.texts #selecting the email data from the csv file
#print(texts) #texts is just the email data

    tokenizer = Tokenizer(num_words=40162,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',lower=True)
tokenizer.fit_on_texts(texts)

sequence_train = tokenizer.texts_to_sequences(texts)#converting the emails to sequences
#print(sequence_train)

word_index = tokenizer.word_index #word index is numbers given to indivisual words in data
#rint(word_index)
print((word_index))
print("done")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



padded the email ,so that all the emails are of the same size when a matrix is created of there word indexes
x_train is the padded array of all email data in integer form ..shape(2182,3590)
y_train is lables array , it has a length of total emails and width of 2 bits - 10/01(0/1)

In [5]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
print("1")
#padding the sequence train , ie making all the matrix same
x_train = pad_sequences(sequence_train)
print(x_train.shape)
print(x_train)
#print("2")
print(np.asarray(labels))#converts all label values of email into a matrix
y_train = to_categorical(np.asarray(labels)) #cheak out doc example 
print(y_train.shape)
print(type(y_train))
#print("3")

1
(1048575, 191)
[[   0    0    0 ...    6    1 8069]
 [   0    0    0 ...   10   46 1830]
 [   0    0    0 ...  380  458 5511]
 ...
 [   0    0    0 ...   27 6548 8747]
 [   0    0    0 ... 2120 2875 1840]
 [   0    0    0 ...  424  393    7]]
[0 0 0 ... 0 0 0]
(1048575, 2)
<class 'numpy.ndarray'>


In [6]:

import gensim 
from gensim.models import Word2Vec

from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors
import gensim.downloader as api
word2vec_model300  = gensim.models.KeyedVectors.load_word2vec_format("D:\Dataset\word2vec-GoogleNews-vectors-master\GoogleNews-vectors-negative300.bin",binary= True)
#print(word2vec_model300 )
print("done")


C:\Users\Akash\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\Akash\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


done


In [7]:
embedding_dim = 300
num_word = 40162
vocab_size = len(word_index)
print(vocab_size)
embedding_matrix = np.zeros((vocab_size , embedding_dim)) #defining size of the embedding matrix
print(embedding_matrix.shape)
for word,i in word_index.items(): 
    #print(word)
    if i>=40162:
        continue
    try:
        embedding_vector =word2vec_model300[word]#getting the word vector for the words
        embedding_matrix[i] = embedding_vector
    except KeyError:
         embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),300)
        #length of all the embedding matrix is 300
print(embedding_matrix[3].shape) #getting a matrix for every word in the dataset

181996
(181996, 300)
(300,)


In [8]:

#print(embedding_matrix)
from keras.layers import Embedding
from keras.initializers import Constant
embedding_layer = Embedding(vocab_size,embedding_dim,embeddings_initializer=Constant(embedding_matrix),trainable=True  )
print(embedding_layer)
#embeddings_initializer=Constant(embedding_matrix),
#embedding_layer.input()1

In [9]:

from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers

sequence_length = x_train.shape[1]
#basically length of the first email (after padding ,so all email shapes are same)
print(x_train[1].shape)
#print(sequence_length)
filter_sizes = [3,4,5]
num_filters = 100

drop = 0.5

print("Done")
print(sequence_length)

(191,)
Done
191


In [10]:
inputs = Input(shape=(sequence_length,))  
print("input layer")
print(keras.backend.shape(inputs))
#embedding_layer = Embedding(vocab_size, embedding_dim, embeddings_initializer=Constant(embedding_matrix), trainable=False)(inputs)
embedding = embedding_layer(inputs)
print("embedding layer")
print(keras.backend.shape(embedding))

reshape = Reshape((sequence_length,embedding_dim,1))(embedding)
print("reshape layer")
print(keras.backend.shape(reshape))

#conv_0 = Conv2D(num_filters, (filter_sizes[0], embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_0 = Conv2D(512, (filter_sizes[0], embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

print("conv0 layer")
print(keras.backend.shape(conv_0))

conv_1 = Conv2D(512, (filter_sizes[1],embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
print('conv1 layer')
print(keras.backend.shape(conv_1))

conv_2 = Conv2D(512, (filter_sizes[2],embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
print("conv2 layer")
print(keras.backend.shape(conv_2))

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
print("maxpool0 layer")
print(keras.backend.shape(maxpool_0))
#flat_0 = Flatten()(maxpool_0)

maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
print("maxpool1 layer")
print(keras.backend.shape(maxpool_1))
#flat_1 = Flatten()(maxpool_1)

maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)
print("maxpool2 layer")
print(keras.backend.shape(maxpool_2))
#flat_2 = Flatten()(maxpool_2)

input layer
Tensor("Shape:0", shape=(2,), dtype=int32)
Instructions for updating:
Colocations handled automatically by placer.
embedding layer
Tensor("Shape_1:0", shape=(3,), dtype=int32)
reshape layer
Tensor("Shape_2:0", shape=(4,), dtype=int32)
conv0 layer
Tensor("Shape_3:0", shape=(4,), dtype=int32)
conv1 layer
Tensor("Shape_4:0", shape=(4,), dtype=int32)
conv2 layer
Tensor("Shape_5:0", shape=(4,), dtype=int32)
maxpool0 layer
Tensor("Shape_6:0", shape=(4,), dtype=int32)
maxpool1 layer
Tensor("Shape_7:0", shape=(4,), dtype=int32)
maxpool2 layer
Tensor("Shape_8:0", shape=(4,), dtype=int32)


In [11]:
merged_tensor = concatenate([maxpool_0,maxpool_1, maxpool_2])
print(keras.backend.shape(merged_tensor))

flatten = Flatten()(merged_tensor)
print(keras.backend.shape(flatten))

#reshape = Reshape((3*num_filters,))(merged_tensor)

dropout = Dropout(drop)(flatten)

output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

print(keras.backend.shape(output))
# this creates a model that includes
model = Model(inputs, output)
print("done")
print(y_train)
print(output)

Tensor("Shape_9:0", shape=(4,), dtype=int32)
Tensor("Shape_10:0", shape=(2,), dtype=int32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Tensor("Shape_11:0", shape=(2,), dtype=int32)
done
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Tensor("dense_1/Softmax:0", shape=(?, 3), dtype=float32)


In [12]:
adam =Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='categorical_crossentropy',  optimizer=adam, metrics=['acc'])

callback = EarlyStopping(monitor='val_loss')
batch_size = 3590
print(model.summary())
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='D:\model_plot.png', show_shapes=True, show_layer_names=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 191)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 191, 300)     54598800    input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 191, 300, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 189, 1, 512)  461312      reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [13]:

model.fit(x_train, y_train, batch_size  ,epochs=100, verbose=1, callbacks = callback )
          # starts training


ValueError: Error when checking target: expected dense_1 to have shape (3,) but got array with shape (2,)